In [1]:
import trade_analysis as ta
import datetime
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import random
from dateutil import parser
import matplotlib.dates as mdates
import re

In [2]:
start_time = datetime.datetime.strptime("14/01/2022-00:00", "%d/%m/%Y-%H:%M").replace(tzinfo=datetime.timezone.utc).timestamp()
end_time = datetime.datetime.utcnow().timestamp()

In [ ]:
df = ta.get_ftx_fills(start_time, end_time)

In [ ]:
df_prices = ta.get_ftx_historical_prices(start_time, end_time)

In [ ]:
df_trades = pd.DataFrame(columns=["entry_time", "entry", "exit_time", "exit", "side", "size", "min", "max", "profit", "drawdown", "drawdown_idx", 
                                  "drawup", "drawup_idx", "drawdown_till_drawup"])

net_profit = 0

idx3 = 0
for idx in range(0, len(df) - 1, 2):
    if df.iloc[idx]["side"] == "buy":
        profit = (df.iloc[idx + 1]["price"] - df.iloc[idx]["price"]) * df.iloc[idx]["size"]
    elif df.iloc[idx]["side"] == "sell":
        profit = (df.iloc[idx]["price"] - df.iloc[idx + 1]["price"]) * df.iloc[idx]["size"]
    net_profit += profit
    
    start_idx, end_idx = 0, 0
    for idx2 in range(len(df_prices)):
        if not start_idx:
            if df_prices.index[idx2] >= df.index[idx].round(freq='5min'):
                start_idx = idx2
        if not end_idx:
            if df_prices.index[idx2] >= df.index[idx + 1].round(freq='5min'):
                end_idx = idx2
                
    if end_idx - start_idx < 3:
        continue
    
    min_value, min_value_idx = df_prices.iloc[start_idx:end_idx]["low"].min(), df_prices.iloc[start_idx:end_idx]["low"].argmin()
    max_value, max_value_idx = df_prices.iloc[start_idx:end_idx]["high"].max(), df_prices.iloc[start_idx:end_idx]["high"].argmax()
    
    max_neg_change = (df.iloc[idx]["price"] - min_value) / min_value * 100
    max_pos_change = (max_value - df.iloc[idx]["price"]) / df.iloc[idx]["price"] * 100
    
    min_value_drawup = df_prices.iloc[start_idx:start_idx + min_value_idx]["low"].min()
    max_value_drawup = df_prices.iloc[start_idx:start_idx + max_value_idx]["high"].max()
    max_neg_change_drawup = (df.iloc[idx]["price"] - min_value_drawup) / min_value_drawup * 100
    max_pos_change_drawup = (max_value_drawup - df.iloc[idx]["price"]) / df.iloc[idx]["price"] * 100
        
    if df.iloc[idx]["side"] == "buy":
        df_trades.loc[idx3] = [df.index[idx], df.iloc[idx]["price"], df.index[idx+1], df.iloc[idx+1]["price"], df.iloc[idx]["side"], df.iloc[idx]["size"], 
                               min_value, max_value, profit, max_neg_change, min_value_idx, max_pos_change, max_value_idx, max_neg_change_drawup]
    elif df.iloc[idx]["side"] == "sell":
        df_trades.loc[idx3] = [df.index[idx], df.iloc[idx]["price"], df.index[idx+1], df.iloc[idx+1]["price"], df.iloc[idx]["side"], df.iloc[idx]["size"], 
                               min_value, max_value, profit, max_pos_change, max_value_idx, max_neg_change, min_value_idx, max_pos_change_drawup]
    idx3 += 1
    
df_trades['pct_change'] = np.where(df_trades.side == "sell", (df_trades.entry - df_trades.exit) / df_trades.exit * 100, 
                                   (df_trades.exit - df_trades.entry) / df_trades.entry * 100)

In [ ]:
df_trades

In [ ]:
df_trades.drawdown_till_drawup.median()

In [ ]:
table = [
    ['', 'Drawdown', 'Drawup', "Pct. Change"], 
    ['Median', df_trades.drawdown.median(), df_trades.drawup.median(), df_trades["pct_change"].median()], 
    ['Mean', df_trades.drawdown.mean(), df_trades.drawup.mean(), df_trades["pct_change"].mean()], 
    ['Max', df_trades.drawdown.max(), df_trades.drawup.max(), df_trades["pct_change"].max()],
    ['Min', df_trades.drawdown.min(), df_trades.drawup.min(), df_trades["pct_change"].min()],
    ['Hit First', (df_trades.drawdown_idx < df_trades.drawup_idx).sum(), (df_trades.drawup_idx < df_trades.drawdown_idx).sum()],
    ['Median*Hit First', df_trades.drawdown.median() * (df_trades.drawdown_idx < df_trades.drawup_idx).sum(), df_trades.drawup.median() * (df_trades.drawup_idx < df_trades.drawdown_idx).sum()]
]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

# Optimizing position close

In [ ]:
drawdown_medians = []
drawup_medians = []

drawdown_means = []
drawup_means = []

drawdown_hit_first = []
drawup_hit_first = []

drawdown_till_drawups = []

net_profit = 0

for _end_idx in range(2, 30):
    df_trades = pd.DataFrame(columns=["entry_time", "entry", "exit_time", "exit", "side", "size", "min", "max", "profit", "drawdown", "drawdown_idx", 
                                  "drawup", "drawup_idx", "drawdown_till_drawup"])

    idx3 = 0
    for idx in range(0, len(df) - 1, 2):
        if df.iloc[idx]["side"] == "buy":
            profit = (df.iloc[idx + 1]["price"] - df.iloc[idx]["price"]) * df.iloc[idx]["size"]
        elif df.iloc[idx]["side"] == "sell":
            profit = (df.iloc[idx]["price"] - df.iloc[idx + 1]["price"]) * df.iloc[idx]["size"]
        net_profit += profit

        start_idx = 0
        for idx2 in range(len(df_prices)):
            if not start_idx:
                if df_prices.index[idx2] >= df.index[idx].round(freq='5min'):
                    start_idx = idx2

        end_idx = start_idx + _end_idx

        min_value, min_value_idx = df_prices.iloc[start_idx:end_idx]["low"].min(), df_prices.iloc[start_idx:end_idx]["low"].argmin()
        max_value, max_value_idx = df_prices.iloc[start_idx:end_idx]["high"].max(), df_prices.iloc[start_idx:end_idx]["high"].argmax()

        max_neg_change = (df.iloc[idx]["price"] - min_value) / min_value * 100
        max_pos_change = (max_value - df.iloc[idx]["price"]) / df.iloc[idx]["price"] * 100

        min_value_drawup = df_prices.iloc[start_idx:start_idx + min_value_idx]["low"].min()
        max_value_drawup = df_prices.iloc[start_idx:start_idx + max_value_idx]["high"].max()
        max_neg_change_drawup = (df.iloc[idx]["price"] - min_value_drawup) / min_value_drawup * 100
        max_pos_change_drawup = (max_value_drawup - df.iloc[idx]["price"]) / df.iloc[idx]["price"] * 100

        if df.iloc[idx]["side"] == "buy":
            df_trades.loc[idx3] = [df.index[idx], df.iloc[idx]["price"], df.index[idx+1], df.iloc[idx+1]["price"], df.iloc[idx]["side"], df.iloc[idx]["size"], 
                                   min_value, max_value, profit, max_neg_change, min_value_idx, max_pos_change, max_value_idx, max_neg_change_drawup]
        elif df.iloc[idx]["side"] == "sell":
            df_trades.loc[idx3] = [df.index[idx], df.iloc[idx]["price"], df.index[idx+1], df.iloc[idx+1]["price"], df.iloc[idx]["side"], df.iloc[idx]["size"], 
                                   min_value, max_value, profit, max_pos_change, max_value_idx, max_neg_change, min_value_idx, max_pos_change_drawup]
        idx3 += 1
    
    drawdown_medians.append(df_trades.drawdown.median())
    drawup_medians.append(df_trades.drawup.median())

    drawdown_means.append(df_trades.drawdown.mean())
    drawup_means.append(df_trades.drawup.mean())
    
    drawdown_hit_first.append((df_trades.drawdown_idx < df_trades.drawup_idx).sum())
    drawup_hit_first.append((df_trades.drawup_idx < df_trades.drawdown_idx).sum())
    
    drawdown_till_drawups.append(df_trades.drawdown_till_drawup.median())

# Optimal results: Two periods (10 mins)

In [ ]:
table = [
    ['', 'Drawdown', 'Drawup', "Pct. Change"], 
    ['Median', df_trades.drawdown.median(), df_trades.drawup.median(), df_trades["pct_change"].median()], 
    ['Mean', df_trades.drawdown.mean(), df_trades.drawup.mean(), df_trades["pct_change"].mean()], 
    ['Max', df_trades.drawdown.max(), df_trades.drawup.max(), df_trades["pct_change"].max()],
    ['Min', df_trades.drawdown.min(), df_trades.drawup.min(), df_trades["pct_change"].min()],
    ['Hit First', (df_trades.drawdown_idx < df_trades.drawup_idx).sum(), (df_trades.drawup_idx < df_trades.drawdown_idx).sum()],
    ['Median*Hit First', df_trades.drawdown.median() * (df_trades.drawdown_idx < df_trades.drawup_idx).sum(), df_trades.drawup.median() * (df_trades.drawup_idx < df_trades.drawdown_idx).sum()]
]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

# Variation with period

In [ ]:
fig, (ax, ax1, ax2, ax3) = plt.subplots(nrows=4, sharex=True, figsize=(15, 10))

x = np.arange(2, 30)

ax.plot(x, drawdown_medians, "o-", label="Drawdown median", color="indianred")
ax.plot(x, drawup_medians, "d-", label="Drawup median", color="forestgreen")
ax.plot(x, drawdown_means, "o-", label="Drawdown mean", color="lightcoral")
ax.plot(x, drawup_means, "d-", label="Drawup mean", color="limegreen")
ax.plot(x, drawdown_till_drawups, "s-", label="Median drawdown till drawup", color="tab:red")
ax.set_ylabel("Percent")
ax.legend()

ax1.plot(x, drawup_hit_first, "o-", label="Drawup hit first", color="forestgreen")
ax1.plot(x, drawdown_hit_first, "o-", label="Drawdown hit first", color="indianred")
ax1.set_ylabel("Hit first")
ax1.legend()

ax2.plot(x, np.array(drawup_hit_first) / np.array(drawdown_hit_first), "o-", color="forestgreen")
ax2.set_ylabel("Draup/Drawdown hit first ratio")

ax3.plot(x, np.array(drawdown_medians) * np.array(drawdown_hit_first), "o-", label="Drawdown median * hit first", color="indianred")
ax3.plot(x, np.array(drawup_medians) * np.array(drawup_hit_first), "o-", label="Drawup median * hit first", color="forestgreen")
ax3.legend()

ax3.set_xlabel("Periods (x5 mins)")

#ax.set_ylim(0.2, 0.6)
#ax.set_xlim(1.5, 3.5)
#fig.savefig("results/2022-01-23_optimal_closing_period_region.jpg", dpi=600)

# Expected return

In [ ]:
drawdown_till_drawups[0]

In [ ]:
inital_capital = 100000
percent_profitable = 25 / (25 + 18) * 100
profit_pct = 0.36
loss_pct = 0.25
num_trades = 100

In [ ]:
def calculate_returns(inital_capital, percent_profitable, profit_pct, loss_pct, num_trades):
    num_profitable = int(num_trades * percent_profitable / 100)
    num_unprofitable = num_trades - num_profitable
    
    profitable_unprofitable_list = ['profitable'] * num_profitable + ['unprofitable'] * num_unprofitable
    
    final_capital = inital_capital

    for n in range(num_trades):
        choice = random.choice(profitable_unprofitable_list)
        if choice == "profitable":
            final_capital += profit_pct/100 * final_capital
        else:
            final_capital -= profit_pct/100 * final_capital
            
    return final_capital, num_profitable, num_unprofitable

In [ ]:
calculate_returns(inital_capital, percent_profitable, profit_pct, loss_pct, num_trades)

In [ ]:
table = [
    ['Inital capital', 'Num trades', 'Num profitable', 'Num unprofitable', "Final capital"], 
    [inital_capital, num_trades, num_profitable, num_unprofitable, final_capital], 
]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

# Funding rate trading

In [ ]:
btc_price = []
whale_position = []
net_position = []
funding_rate = []
times = []

with open("whale-trades-bot.log", 'r') as f:
    for line in f:
        if "Whale" in line:
            match = re.findall("-?\d*\.?\d+", line)
            times.append(parser.parse(line[:29]))
            btc_price.append(float(match[9]))
            whale_position.append(float(match[10]))
            net_position.append(float(match[11]))
            funding_rate.append(float(match[12]) * 10**int(match[13]))

In [ ]:
fig, (ax1, ax3) = plt.subplots(nrows=2, figsize=(15, 10))

ax1.plot(times, whale_position, label="Whale")
ax1.plot(times, net_position, label="Net")
ax1.legend()
ax1.axhline(10, linestyle="--", color="tab:grey")
ax1.axhline(-10, linestyle="--", color="tab:grey")

ax2 = ax1.twinx()
ax2.plot(times, btc_price, color="tab:red")
ax2.tick_params(axis='y', labelcolor="tab:red")

ax3.plot(times, funding_rate, ".-")

ax1.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax1.xaxis.get_major_locator()))

for idx in range(len(funding_rate) - 1):
    current_rate = funding_rate[idx] + 1
    next_rate = funding_rate[idx + 1] + 1
    funding_rate_change = abs(current_rate - next_rate) / next_rate * 100
    if funding_rate_change > 4e-4:
        if current_rate > next_rate:
            ax3.axvline(times[idx], linestyle="--", color="tab:red")
            ax1.axvline(times[idx], linestyle="--", color="tab:red")
        else:
            ax3.axvline(times[idx], linestyle="--", color="tab:green")
            ax1.axvline(times[idx], linestyle="--", color="tab:green")